In [3]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd    
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.svm import LinearSVC, SVC, NuSVC, libsvm, SVR
from sklearn.metrics import  accuracy_score
from sklearn import metrics
from sklearn.utils import shuffle
import numpy as np

In [4]:
train=pd.read_csv("train.csv", header=None)
train = np.array(shuffle(train))

test=pd.read_csv("test.csv", header=None)
test = np.array(shuffle(test))

In [5]:
print("Train size:",len(train), "Test size:", len(test))

Train size: 1239 Test size: 315


In [6]:
train_sum = []
for i in range(len(train)):
    s=0
    for j in range(15,26): 
        s+=train[i][j]
    train_sum.append(s)

test_sum = []
for i in range(len(test)):
    s=0
    for j in range(15,26):
        s+=test[i][j]
    test_sum.append(s)

In [7]:
train_text = [i[2] for i in train]
test_text = [i[2] for i in test]

In [8]:
def get_features(train_arr, test_arr):
    vectorizer = CountVectorizer()
    vectorizer.fit(train_arr)
    train_mat = vectorizer.transform(train_arr)

    tfidf = TfidfTransformer()
    tfidf.fit(train_mat)
    train_tfmat = tfidf.transform(train_mat)

    test_mat = vectorizer.transform(test_arr)
    test_tfmat = tfidf.transform(test_mat)
    return train_tfmat, test_tfmat

In [37]:
print("Two classes \nScore > 0 Yes\nScore < 0 No")
binary_train_labels = ["Yes" if i>0 else "No" for i in train_sum]
binary_test_labels = ["Yes" if i>0 else "No" for i in test_sum]

train_lbl = binary_train_labels
test_lbl = binary_test_labels

train_tfmat, test_tfmat = get_features(train_text, test_text)

print("\nTraining SVM model...")
lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)
y_pred_lsvm=lsvm.predict(test_tfmat)
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

Two classes 
Score > 0 Yes
Score < 0 No

Training SVM model...
accuracy: 0.8126984126984127


In [38]:
print("Train: Yes:",binary_train_labels.count("Yes"),", No:",binary_train_labels.count("No"))
print("Test: Yes:",binary_test_labels.count("Yes"),", No:",binary_test_labels.count("No"))

Train: Yes: 1014 , No: 225
Test: Yes: 256 , No: 59


In [39]:
splitter = 3
print("Three classes \n["+str(splitter+1)+",11]-> Yes \n[-"+str(splitter)+","+str(splitter)+"]-> Conflict \n[-"+str(splitter+1)+",-11]-> No")
three_train_labels=[]
for i in train_sum:
    if i>splitter:
        three_train_labels.append("Yes")
    elif i<=splitter and i>=-splitter:
        three_train_labels.append("Conflict")
    elif i<-splitter:
        three_train_labels.append("No")
        
three_test_labels=[]
for i in test_sum:
    if i>splitter:
        three_test_labels.append("Yes")
    elif i<=splitter and i>=-splitter:
        three_test_labels.append("Conflict")
    elif i<-splitter:
        three_test_labels.append("No")
        
train_lbl = three_train_labels
test_lbl = three_test_labels

train_tfmat, test_tfmat = get_features(train_text, test_text)

print("\nTraining SVM model...")
lsvm=LinearSVC(class_weight="balanced")
lsvm.fit(train_tfmat,train_lbl)
y_pred_lsvm=lsvm.predict(test_tfmat)
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

Three classes 
[4,11]-> Yes 
[-3,3]-> Conflict 
[-4,-11]-> No

Training SVM model...
accuracy: 0.5968253968253968


In [12]:
print("Train: Yes:",three_train_labels.count("Yes"),", Conflict:",three_train_labels.count("Conflict"),", No:",three_train_labels.count("No"))
print("Test: Yes:",three_test_labels.count("Yes"),", Conflict:",three_test_labels.count("Conflict"),", No:",three_test_labels.count("No"))

Train: Yes: 703 , Conflict: 483 , No: 53
Test: Yes: 177 , Conflict: 123 , No: 15


In [33]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0)

print("Two classes \nScore > 0 Yes\nScore < 0 No")

binary_train_labels = ["Yes" if i>0 else "No" for i in train_sum]
binary_test_labels = ["Yes" if i>0 else "No" for i in test_sum]

train_lbl = binary_train_labels
test_lbl = binary_test_labels

train_yes = [(t,i) for t,i in zip(train_text, binary_train_labels)  if "Yes" in i]
train_no = [(t,i) for t,i in zip(train_text, binary_train_labels)  if "No" in i]

print("train_yes:",len(train_yes), "train_no:",len(train_no))

train_yes_text = [i[0] for i in train_yes]
train_yes_lbl = [i[1] for i in train_yes]

train_no_text = [i[0] for i in train_no]
train_no_lbl = [i[1] for i in train_no]

train_tfmat_yes, test_tfmat = get_features(train_yes_text, train_yes_lbl)

print("\nTraining SVM model...")
lsvm=LinearSVC(class_weight="balanced")
lsvm.fit(train_tfmat_yes, train_yes_lbl)
train_tfmat_no, test_tfmat = get_features(train_no_text, train_no_lbl)
lsvm.fit(train_tfmat_no, train_no_lbl)

y_pred_lsvm=lsvm.predict(test_tfmat)
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))



Two classes 
Score > 0 Yes
Score < 0 No
train_yes: 1014 train_no: 225

Training SVM model...


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'Yes'

<1239x9552 sparse matrix of type '<class 'numpy.float64'>'
	with 18412 stored elements in Compressed Sparse Row format>